In [1]:
import os
import sys
from pathlib import Path


try:
    project_root = Path(__file__).resolve().parents[1]
except NameError:
    project_root = Path.cwd().parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"✓ Added project root to sys.path: {project_root}")

# Now you can import from 'src'
from src.duckdb_config import DuckDBConnection

✓ Added project root to sys.path: /home/lliverani/projects/mylife-in-data/data_exploration


In [2]:
# Cell 2: Test Connection
# Create connection
db = DuckDBConnection(database=":memory:")
con = db.connect()

# Test query
result = con.execute("SELECT 'Hello from DuckDB!' as message").df()
print(result)

✓ Loaded extension: httpfs
✓ Loaded extension: parquet
✓ Configured S3/MinIO: localhost:9000
              message
0  Hello from DuckDB!


In [6]:
bucket_name = os.getenv('MINIO_BUCKET')


try:
    s3_files = con.execute(f"SELECT * FROM glob('s3://{bucket_name}/**')").df()
    display(s3_files)
except Exception as e:
    print(f"Error listing S3 files: {e}")

,file
0,s3://inbound/raw/spotify/api/daily/2025-10-20....
1,s3://inbound/raw/spotify/api/daily/2025-10-21....
2,s3://inbound/raw/spotify/api/daily/2025-10-24....
3,s3://inbound/raw/spotify/api/daily/2025-10-25....
4,s3://inbound/raw/spotify/api/daily/2025-10-26....
5,s3://inbound/raw/spotify/api/tracks/2025-10-26...
6,s3://inbound/raw/spotify/download/my_spotify_d...
7,s3://inbound/raw/spotify/download/my_spotify_d...
8,s3://inbound/raw/spotify/download/my_spotify_d...
9,s3://inbound/raw/spotify/download/my_spotify_d...


In [18]:
from src.duckdb_config import explore

# Define the S3 path to your JSONL files using a glob pattern
s3_path = "s3://inbound/raw/spotify/api/tracks/*.jsonl.gz"



# DuckDB will infer the newline-delimited format and handle decompression.
query = f"""
    SELECT 
        raw_item.played_at,
        raw_item.track.uri
    FROM read_json_auto('{s3_path}')

    group by 1,2
    having count(*) > 1
"""

df = explore(query)

# Use itables for interactive display in a notebook
display(df)


✓ Loaded extension: httpfs
✓ Loaded extension: parquet
✓ Configured S3/MinIO: localhost:9000


,played_at,uri


In [10]:
from src.duckdb_config import explore

# Define the S3 path to your JSONL files using a glob pattern
s3_path = "s3://inbound/raw/spotify/api/tracks/*.jsonl.gz"



# DuckDB will infer the newline-delimited format and handle decompression.
query = f"""
    SELECT 
        *
    FROM read_json_auto('{s3_path}')
    ORDER BY _ingestion_metadata.ingested_at desc
"""
# where _ingestion_metadata.run_id not like '%manual%'
df = explore(query)

# Use itables for interactive display in a notebook
display(df)


✓ Loaded extension: httpfs
✓ Loaded extension: parquet
✓ Configured S3/MinIO: localhost:9000


,raw_item,_ingestion_metadata
0,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562119+00...
1,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562110+00...
2,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562100+00...
3,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562091+00...
4,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562080+00...
...,...,...
261,"{'track': {'album': {'album_type': 'single', '...","{'ingested_at': '2025-10-20T00:29:47.866Z', 'e..."
262,"{'track': {'album': {'album_type': 'single', '...","{'ingested_at': '2025-10-20T00:27:36.288Z', 'e..."
263,"{'track': {'album': {'album_type': 'single', '...","{'ingested_at': '2025-10-20T00:25:08.325Z', 'e..."
264,"{'track': {'album': {'album_type': 'single', '...","{'ingested_at': '2025-10-20T00:20:57.690Z', 'e..."


In [15]:
from src.duckdb_config import explore

# Define the S3 path to your JSONL files using a glob pattern
s3_path = "s3://inbound/raw/spotify/api/tracks/*.jsonl.gz"



# DuckDB will infer the newline-delimited format and handle decompression.
query = f"""
    SELECT 
        *
    FROM read_json_auto('{s3_path}')
    where _ingestion_metadata.run_id like '%manual%'
    limit 1
"""
# where _ingestion_metadata.run_id not like '%manual%'
df_manual = explore(query)

# Use itables for interactive display in a notebook
display(df_manual)


✓ Loaded extension: httpfs
✓ Loaded extension: parquet
✓ Configured S3/MinIO: localhost:9000


,raw_item,_ingestion_metadata
0,"{'track': {'album': {'album_type': 'single', '...","{'ingested_at': '2025-10-20T16:18:33.522Z', 'e..."


In [16]:
from src.duckdb_config import explore

# Define the S3 path to your JSONL files using a glob pattern
s3_path = "s3://inbound/raw/spotify/api/tracks/*.jsonl.gz"



# DuckDB will infer the newline-delimited format and handle decompression.
query = f"""
    SELECT 
        *
    FROM read_json_auto('{s3_path}')
    where _ingestion_metadata.run_id not like '%manual%'
    limit 1
"""
# where _ingestion_metadata.run_id not like '%manual%'
df_schedule = explore(query)

# Use itables for interactive display in a notebook
display(df_schedule)


✓ Loaded extension: httpfs
✓ Loaded extension: parquet
✓ Configured S3/MinIO: localhost:9000


,raw_item,_ingestion_metadata
0,"{'track': {'album': {'album_type': 'single', '...",{'ingested_at': '2025-10-26T16:00:01.562017+00...


In [19]:
# The '**' makes it recursive. Change '/**' to a specific path if needed.
path_to_parquet_files = f's3://{bucket_name}/**/*.parquet'


try:
    # DuckDB's read_parquet function can read multiple files at once
    df_parquet = con.execute(f"""
                                 SELECT 
                                 *   
                                 FROM read_parquet('{path_to_parquet_files}')
                                 WHERE track.album.album_type IS NOT NULL
                    
                                 """).df()
   
    print("✓ Successfully loaded Parquet files into DataFrame.")
    display(df_parquet) # Display the first few rows
    
except Exception as e:
    print(f"Error reading Parquet files: {e}")

✓ Successfully loaded Parquet files into DataFrame.


,track,played_at,context,played_at_dt,played_at_date,__index_level_0__
0,"{'album': {'album_type': 'single', 'artists': ...",2025-10-20T16:18:33.522Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-20 16:18:33.522000+00:00,2025-10-20,34
1,"{'album': {'album_type': 'album', 'artists': [...",2025-10-20T16:15:32.878Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-20 16:15:32.878000+00:00,2025-10-20,35
2,"{'album': {'album_type': 'single', 'artists': ...",2025-10-20T16:12:17.379Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-20 16:12:17.379000+00:00,2025-10-20,36
3,"{'album': {'album_type': 'single', 'artists': ...",2025-10-20T16:10:27.851Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-20 16:10:27.851000+00:00,2025-10-20,37
4,"{'album': {'album_type': 'single', 'artists': ...",2025-10-20T08:35:15.813Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-20 08:35:15.813000+00:00,2025-10-20,38
...,...,...,...,...,...,...
246,"{'album': {'album_type': 'single', 'artists': ...",2025-10-26T15:09:41.330Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-26 15:09:41.330000+00:00,2025-10-26,2
247,"{'album': {'album_type': 'single', 'artists': ...",2025-10-26T15:07:38.191Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-26 15:07:38.191000+00:00,2025-10-26,3
248,"{'album': {'album_type': 'single', 'artists': ...",2025-10-26T15:05:46.793Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-26 15:05:46.793000+00:00,2025-10-26,4
249,"{'album': {'album_type': 'single', 'artists': ...",2025-10-26T15:03:45.329Z,{'external_urls': {'spotify': 'https://open.sp...,2025-10-26 15:03:45.329000+00:00,2025-10-26,5


In [89]:
# The '**' makes it recursive. Change '/**' to a specific path if needed.
path_to_parquet_files = f's3://{bucket_name}/raw/spotify/api/daily/*.parquet'


try:
    # DuckDB's read_parquet function can read multiple files at once
    df_api_history = con.execute(f"""
                                 SELECT 
                                 STRFTIME(CAST("played_at_dt" AS TIMESTAMP), '%Y-%m-%d %H:%M:%S') AS played_at_ts,
                                 track.name AS track_name,
                                 track.album.name AS album_name,
                                 track.artists[1].name as artist_name,
                                 track.uri AS spotify_track_uri,
                                 track.duration_ms AS duration_ms,   
                                 FROM read_parquet('{path_to_parquet_files}')
                                 """).df()
   
    print("✓ Successfully loaded Parquet files into DataFrame.")
    display(df_api_history) # Display the first few rows
    
except Exception as e:
    print(f"Error reading Parquet files: {e}")

✓ Successfully loaded Parquet files into DataFrame.


,played_at_ts,track_name,album_name,artist_name,spotify_track_uri,duration_ms
0,2025-10-20 16:18:33,Man I Need,Man I Need,Olivia Dean,spotify:track:1qbmS6ep2hbBRaEZFpn7BX,184000
1,2025-10-20 16:15:32,Outro Lugar,Outro Lugar,Toco,spotify:track:11NkxJtowfjaJksJbjYAS0,201375
2,2025-10-20 16:12:17,Eu Pensei,Eu Pensei,Pedro Mizutani,spotify:track:0OUQ1NJxeDkAEt7LHTFiel,132760
3,2025-10-20 16:10:27,Eu Pensei,Eu Pensei,Pedro Mizutani,spotify:track:0OUQ1NJxeDkAEt7LHTFiel,132760
4,2025-10-20 08:35:15,Fogueira,Tudo Passa,Bagua Records,spotify:track:21TS6QtxgAvBgHGj06SpJ9,221714
...,...,...,...,...,...,...
127,2025-10-25 17:32:18,Sandman,Bad Behavior,BRONCHO,spotify:track:28I0KNqOmkvzzz7SghMhfu,192255
128,2025-10-25 17:57:04,Hush Fires,Hush Fires,Evening Elephants,spotify:track:47C4EneyFv6tJMGGToGz8L,180000
129,2025-10-25 17:54:08,Timezones,The Unsure Mixtape,Schur,spotify:track:5aSeyhSp2qxYr8yBCtPNvr,212517
130,2025-10-25 17:50:34,Testarossa,Testarossa,Schur,spotify:track:6w6V1sgkhxxoM2I4ndfrCZ,126000


In [ ]:
import pprint
import json

track = df_parquet.iat[28, 0]
# pprint.pprint(track)

with open("./data/track.json", 'w') as f:
    json.dump(track, f, indent=4)



In [87]:
# The '**' makes it recursive. Change '/**' to a specific path if needed.
path_to_json_files = f's3://{bucket_name}/raw/spotify/download/**.json'

path_to_json_files = f's3://{bucket_name}/raw/spotify/download/my_spotify_data_extended_history/Spotify Extended Streaming History/Streaming_History_Audio_*.json'

try:

    df_download_history = con.execute(f"""
                                    SELECT 
                                        CAST("ts" AS TIMESTAMP) AS played_at_ts,
                                        master_metadata_track_name AS track_name,
                                        master_metadata_album_album_name AS album_name,                  
                                        master_metadata_album_artist_name AS artist_name,
                                        spotify_track_uri,
                                        ms_played AS duration_ms


                                    FROM read_json_auto('{path_to_json_files}')
                                        WHERE spotify_track_uri IS NOT NULL
                                    """).df()
   
    print("✓ Successfully loaded files into DataFrame.")
    display(df_download_history) # Display the first few rows
    
except Exception as e:
    print(f"Error reading files: {e}")

✓ Successfully loaded files into DataFrame.


,played_at_ts,track_name,album_name,artist_name,spotify_track_uri,duration_ms
0,2020-02-11 16:07:48,Sirvia,Ok Computer Però Trap,Pippo Sowlo,spotify:track:2Y9JMc04myJYtrESxloIWk,10913
1,2020-02-11 16:12:00,Condorello,Condorello,Pippo Sowlo,spotify:track:1kB26kzGkRZZOqXCtA1Lee,251608
2,2020-02-11 17:27:37,La Mia Hit (feat. Max Pezzali),ReAle,J-AX,spotify:track:4bMfWDvKPCFQ2df7Qf2UTs,11958
3,2020-02-11 17:31:04,Mainstream (La scala sociale del Rap),ReAle,J-AX,spotify:track:4SLeGGSSS4ZgreK0HaN9h8,206280
4,2020-02-11 17:33:54,Supercalifragili (feat. Annalisa & Luca Di Ste...,ReAle,J-AX,spotify:track:6ExLQlakJDGUxN5O567dAt,170133
...,...,...,...,...,...,...
35444,2025-10-12 16:41:59,counting the days until we meet again,counting the days until we meet again,sad notes,spotify:track:7jcEaXD19od05Y4aqgffgW,135631
35445,2025-10-12 16:43:34,ValleyViews,ValleyViews,Mellow Mirror,spotify:track:7E2GF1dct7xy9GKgp4eQrj,99031
35446,2025-10-13 18:07:40,Loft Room,Loft Room,Rainbows & Clouds,spotify:track:6uDcf4yMECdvrTVUc06Gfn,97278
35447,2025-10-13 18:12:15,Falling For You,Falling For You,Amelie Lens,spotify:track:6KmjD9CaophLMXTCzKoLLZ,234997
